# Music Record Labels
ETL, XML, Consuming API, SQL

# Task: Use Discogs’ publicly available data to build an ETL pipeline (in python) that creates a table of music label information.

# What will the application do?

## Extract data from the monthly XML dump file of all labels into a pandas DataFrame

In [1]:
import pandas as pd
import time
import psycopg2
import requests


In [3]:
def extract():
    # Read the XML file and convert it into a DataFrame
    print('Begin reading new label releases from discogs.com')
    df = pd.read_xml('C:\\Users\\paperspace\\Downloads\\MusicLabel\\discogs_20230101_labels.xml', xpath='//labels/label')
    print(f'Successfully loaded {len(df)} labels into DataFrame')
    return df
try1 = extract()
try1

Begin reading new label releases from discogs.com
Successfully loaded 1946246 labels into DataFrame


,images,id,name,contactinfo,profile,data_quality,urls,sublabels,parentLabel
0,NaN,1,Planet E,Planet E Communications\r\nP.O. Box 27218\r\nD...,[a=Carl Craig]'s classic techno label founded ...,Needs Vote,NaN,NaN,None
1,NaN,2,Earthtones Recordings,Seasons Recordings\r\n2236 Pacific Avenue\r\nS...,California deep house label founded by [a=Jami...,Correct,NaN,NaN,None
2,NaN,3,Seasons Recordings,"Seasons Recordings\r\nCosta Mesa, CA 92627\r\n...",California deep-house label founded by [a=Jami...,Needs Vote,NaN,NaN,None
3,NaN,4,Siesta Music,Siesta Records\r\n1913 Via Encantadoras\r\nSan...,None,Needs Vote,NaN,NaN,None
4,NaN,5,Svek,Svek office \r\nStephan Grieder \r\nFax: +46 (...,None,Correct,NaN,NaN,Goldhead Music
...,...,...,...,...,...,...,...,...,...
1946241,NaN,3078049,5th Gear Productions,None,None,Needs Major Changes,NaN,NaN,None
1946242,NaN,3078052,"Tunnell, Jimi Music",None,None,Needs Major Changes,NaN,NaN,None
1946243,NaN,3078055,Warrior Gospel Records,None,None,Needs Major Changes,NaN,NaN,None
1946244,NaN,3078058,Allegretto Music,None,None,Needs Major Changes,NaN,NaN,None


Consumer Key	BlLxXNiHJoVLBApblBoO

Consumer Secret	grxJeFlhvKDYugDExZeNfEyegmudzdLY

Request Token URL	https://api.discogs.com/oauth/request_token

Authorize URL	https://www.discogs.com/oauth/authorize

Access Token URL	https://api.discogs.com/oauth/access_token

In [3]:
pip install discogs_client


     -------------------------------------- 151.7/151.7 kB 4.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import discogs_client

# Replace with your own consumer key and secret
consumer_key = 'BlLxXNiHJoVLBApblBoO'
consumer_secret = 'grxJeFlhvKDYugDExZeNfEyegmudzdLY'

# Create a Discogs client object
discogs = discogs_client.Client('YourApp/1.0', consumer_key=consumer_key, consumer_secret=consumer_secret)

# Get the OAuth request token
request_token, request_secret, authorize_url = discogs.get_authorize_url()

# Print a message with instructions for manual authorization
print("Please authorize the application by visiting the following URL:")
print(authorize_url)


Please authorize the application by visiting the following URL:
https://www.discogs.com/oauth/authorize?oauth_token=IWytxhUoMqDUZENhjaQNuFeiPAqvmRrQSoCswySl


In [6]:
# After authorizing the application, Discogs will redirect you to a callback URL.
# Extract the `oauth_verifier` parameter from the URL.
# You can manually copy it or programmatically retrieve it using a library like Flask.

# Once you have the `oauth_verifier`, you can authenticate the client.
oauth_verifier = 'KpGZCXNggK'  # Replace with the actual oauth_verifier value
access_token, access_secret = discogs.get_access_token(oauth_verifier)

# Set the access token and secret for further API requests
discogs.set_token(access_token, access_secret)

# Now you can make authenticated requests to the Discogs API
# For example, to get your user identity:
user = discogs.identity()
print("Authenticated user:", user.username)


Authenticated user: nirmala_joshi5


# In accordance with the Build Specifications below
Additionally by using the Discogs API to add the following additional columns:

Total number of releases: How many total releases does the label have?

Minimum release year: What year was the label’s first release?

Maximum release year: What year was the label’s last release?

    Remove any labels where: (1 Point)
    Data quality is not listed as “Complete and Correct”
    Contact information is missing


In [4]:

def get_label_info(label_id):
    base_url = 'https://api.discogs.com/labels/'
    label_url = base_url + str(label_id) + '/releases'

    try:
        response = requests.get(label_url)
        if response.status_code == 200:
            label_info = response.json()
            return label_info
        else:
            print(f"{time.strftime('%H:%M:%S')} | Failed API call for label {label_id}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"{time.strftime('%H:%M:%S')} | Failed API call for label {label_id}: {e}")
        return None


In [5]:
def add_additional_columns(df):
    print(f"{time.strftime('%H:%M:%S')}|Begin API calls for additional label info ({len(df)} labels)")
    failed_labels = []

    df['num_of_releases'] = 0  # Initialize 'num_of_releases' column with zeros

    for index, row in df.iterrows():
        label_id = row['id']
        label_info = get_label_info(label_id)

        if label_info is not None:
            release_years = [release['year'] for release in label_info['releases']]
            if release_years:
                max_year = max(release_years)
                min_year = min(release_years)
            else:
                max_year = None
                min_year = None

            # Update the 'num_of_releases' column with the count
#             df.at[index, 'num_of_releases'] = len(label_info['releases'])
            df.at[index, 'num_of_releases'] = label_info['pagination']['item'])
            # Add additional columns to the DataFrame
            df.at[index, 'min_release_year'] = min_year
            df.at[index, 'max_release_year'] = max_year
        else:
            failed_labels.append(label_id)

        # Implement rate limiting to avoid hitting the API rate limit
        time.sleep(2)  # Adjust the sleep duration as needed

    if failed_labels:
        print(f"{time.strftime('%H:%M:%S')}|Failed API calls for labels: {', '.join(str(l) for l in failed_labels)}")

#     df['num_of_releases'] = df['num_of_releases'].astype(int)  # Convert 'num_of_releases' column to int
#     df['min_release_year'] = df['min_release_year'].fillna(0).astype(int)
#     df['max_release_year'] = df['max_release_year'].fillna(0).astype(int)

    return df


In [ ]:
# Transform data

In [8]:
def transform(df):
    
    print('Begin cleaning columns')
    df = df[~df['data_quality'].isin(['Needs Major Changes', 'Needs Vote', 'Correct', 'Needs Minor Changes', 'Entirely Incorrect'])]
    df = df[df['contactinfo'].notna()]
    new_dfs = []

    for label_id in df['id']:
        try:
            new_df = add_additional_columns(pd.DataFrame([{'id': label_id}]))
            new_dfs.append(new_df)
        except KeyError:
            print(f"Failed to add additional columns for label {label_id}. Skipping...")
            continue
    final_new_df = pd.concat([df[['id', 'name', 'contactinfo', 'profile','parentLabel']].reset_index(drop=True), pd.concat(new_dfs).reset_index(drop=True)[['num_of_releases', 'max_release_year', 'min_release_year']]], axis=1).copy()
    final_new_df['num_of_releases'] = final_new_df['num_of_releases'].fillna(0).astype(int)  # Convert 'num_of_releases' column to int
    final_new_df['min_release_year'] = final_new_df['min_release_year'].fillna(0).astype(int)
    final_new_df['max_release_year'] = final_new_df['max_release_year'].fillna(0).astype(int)
    print(f'Completed cleaning {len(final_new_df)}columns')   
    return final_new_df

try2   = transform(try1)
try2 

Begin cleaning columns
12:27:59|Begin API calls for additional label info (1 labels)
12:28:03|Begin API calls for additional label info (1 labels)
12:28:05|Begin API calls for additional label info (1 labels)
12:28:07|Begin API calls for additional label info (1 labels)
12:28:09|Begin API calls for additional label info (1 labels)
12:28:12|Begin API calls for additional label info (1 labels)
12:28:14|Begin API calls for additional label info (1 labels)
12:28:16|Begin API calls for additional label info (1 labels)
12:28:19|Begin API calls for additional label info (1 labels)
12:28:21|Begin API calls for additional label info (1 labels)
12:28:23|Begin API calls for additional label info (1 labels)
12:28:26|Begin API calls for additional label info (1 labels)
12:28:29|Begin API calls for additional label info (1 labels)
12:28:31|Begin API calls for additional label info (1 labels)
12:28:34|Begin API calls for additional label info (1 labels)
12:28:36|Begin API calls for additional label i

12:30:44|Failed API calls for labels: 30640
12:30:44|Begin API calls for additional label info (1 labels)
12:30:44 | Failed API call for label 31387
12:30:46|Failed API calls for labels: 31387
12:30:46|Begin API calls for additional label info (1 labels)
12:30:46 | Failed API call for label 31616
12:30:48|Failed API calls for labels: 31616
12:30:48|Begin API calls for additional label info (1 labels)
12:30:49 | Failed API call for label 31723
12:30:51|Failed API calls for labels: 31723
12:30:51|Begin API calls for additional label info (1 labels)
12:30:51 | Failed API call for label 33443
12:30:53|Failed API calls for labels: 33443
12:30:53|Begin API calls for additional label info (1 labels)
12:30:53 | Failed API call for label 33579
12:30:55|Failed API calls for labels: 33579
12:30:55|Begin API calls for additional label info (1 labels)
12:30:55 | Failed API call for label 34271
12:30:57|Failed API calls for labels: 34271
12:30:57|Begin API calls for additional label info (1 labels)


12:32:45|Failed API calls for labels: 82680
12:32:45|Begin API calls for additional label info (1 labels)
12:32:45 | Failed API call for label 84175
12:32:47|Failed API calls for labels: 84175
12:32:47|Begin API calls for additional label info (1 labels)
12:32:47 | Failed API call for label 84691
12:32:49|Failed API calls for labels: 84691
12:32:49|Begin API calls for additional label info (1 labels)
12:32:49 | Failed API call for label 84829
12:32:51|Failed API calls for labels: 84829
12:32:51|Begin API calls for additional label info (1 labels)
12:32:51 | Failed API call for label 86130
12:32:53|Failed API calls for labels: 86130
12:32:53|Begin API calls for additional label info (1 labels)
12:32:53 | Failed API call for label 86362
12:32:55|Failed API calls for labels: 86362
12:32:55|Begin API calls for additional label info (1 labels)
12:32:56 | Failed API call for label 87599
12:32:58|Failed API calls for labels: 87599
12:32:58|Begin API calls for additional label info (1 labels)


12:34:46 | Failed API call for label 187159
12:34:48|Failed API calls for labels: 187159
12:34:48|Begin API calls for additional label info (1 labels)
12:34:48 | Failed API call for label 188105
12:34:50|Failed API calls for labels: 188105
12:34:50|Begin API calls for additional label info (1 labels)
12:34:51 | Failed API call for label 192475
12:34:53|Failed API calls for labels: 192475
12:34:53|Begin API calls for additional label info (1 labels)
12:34:53 | Failed API call for label 193726
12:34:55|Failed API calls for labels: 193726
12:34:55|Begin API calls for additional label info (1 labels)
12:34:55 | Failed API call for label 195143
12:34:57|Failed API calls for labels: 195143
12:34:57|Begin API calls for additional label info (1 labels)
12:34:58 | Failed API call for label 196567
12:35:00|Failed API calls for labels: 196567
12:35:00|Begin API calls for additional label info (1 labels)
12:35:00 | Failed API call for label 198453
12:35:02|Failed API calls for labels: 198453
12:35

12:36:47|Failed API calls for labels: 348515
12:36:47|Begin API calls for additional label info (1 labels)
12:36:47 | Failed API call for label 348638
12:36:49|Failed API calls for labels: 348638
12:36:49|Begin API calls for additional label info (1 labels)
12:36:50 | Failed API call for label 351479
12:36:52|Failed API calls for labels: 351479
12:36:52|Begin API calls for additional label info (1 labels)
12:36:52 | Failed API call for label 361078
12:36:54|Failed API calls for labels: 361078
12:36:54|Begin API calls for additional label info (1 labels)
12:36:54 | Failed API call for label 361619
12:36:56|Failed API calls for labels: 361619
12:36:56|Begin API calls for additional label info (1 labels)
12:36:56 | Failed API call for label 378105
12:36:58|Failed API calls for labels: 378105
12:36:58|Begin API calls for additional label info (1 labels)
12:36:58 | Failed API call for label 380955
12:37:00|Failed API calls for labels: 380955
12:37:00|Begin API calls for additional label inf

12:38:46 | Failed API call for label 625856
12:38:48|Failed API calls for labels: 625856
12:38:48|Begin API calls for additional label info (1 labels)
12:38:48 | Failed API call for label 630118
12:38:50|Failed API calls for labels: 630118
12:38:50|Begin API calls for additional label info (1 labels)
12:38:50 | Failed API call for label 644713
12:38:52|Failed API calls for labels: 644713
12:38:52|Begin API calls for additional label info (1 labels)
12:38:52 | Failed API call for label 649914
12:38:54|Failed API calls for labels: 649914
12:38:54|Begin API calls for additional label info (1 labels)
12:38:55 | Failed API call for label 652085
12:38:57|Failed API calls for labels: 652085
12:38:57|Begin API calls for additional label info (1 labels)
12:38:57 | Failed API call for label 660151
12:38:59|Failed API calls for labels: 660151
12:38:59|Begin API calls for additional label info (1 labels)
12:38:59 | Failed API call for label 662512
12:39:01|Failed API calls for labels: 662512
12:39

12:40:47|Failed API calls for labels: 984355
12:40:47|Begin API calls for additional label info (1 labels)
12:40:47 | Failed API call for label 1003430
12:40:49|Failed API calls for labels: 1003430
12:40:49|Begin API calls for additional label info (1 labels)
12:40:49 | Failed API call for label 1010433
12:40:51|Failed API calls for labels: 1010433
12:40:51|Begin API calls for additional label info (1 labels)
12:40:51 | Failed API call for label 1037282
12:40:53|Failed API calls for labels: 1037282
12:40:53|Begin API calls for additional label info (1 labels)
12:40:53 | Failed API call for label 1040376
12:40:55|Failed API calls for labels: 1040376
12:40:55|Begin API calls for additional label info (1 labels)
12:40:56 | Failed API call for label 1041572
12:40:58|Failed API calls for labels: 1041572
12:40:58|Begin API calls for additional label info (1 labels)
12:40:58 | Failed API call for label 1049898
12:41:00|Failed API calls for labels: 1049898
12:41:00|Begin API calls for addition

12:42:44|Failed API calls for labels: 1761782
12:42:44|Begin API calls for additional label info (1 labels)
12:42:44 | Failed API call for label 1763663
12:42:46|Failed API calls for labels: 1763663
12:42:46|Begin API calls for additional label info (1 labels)
12:42:47 | Failed API call for label 1795801
12:42:49|Failed API calls for labels: 1795801
12:42:49|Begin API calls for additional label info (1 labels)
12:42:49 | Failed API call for label 1798608
12:42:51|Failed API calls for labels: 1798608
12:42:51|Begin API calls for additional label info (1 labels)
12:42:51 | Failed API call for label 1800956
12:42:53|Failed API calls for labels: 1800956
12:42:53|Begin API calls for additional label info (1 labels)
12:42:53 | Failed API call for label 1806978
12:42:55|Failed API calls for labels: 1806978
12:42:55|Begin API calls for additional label info (1 labels)
12:42:55 | Failed API call for label 1844488
12:42:57|Failed API calls for labels: 1844488
12:42:57|Begin API calls for additio

,id,name,contactinfo,profile,parentLabel,num_of_releases,max_release_year,min_release_year
0,774,Hed Kandi Records,"MSHK Limited\r\n103 Gaunt Street\r\nLondon, SE...",Record company of the UK-based house label [l6...,Ministry Of Sound,50,2015,0
1,3141,Concept,Concept Records\r\n24A Barry Street\r\nBradfor...,Trance label from the UK. The label becomes [l...,Global Beat Records,4,1996,1996
2,3203,Tinnitus Tonträger,Tinnitus Tonträger\r\nMarc Göttling\r\nWiesent...,Style: hard kicking minimal techno traxx.,None,11,2002,0
3,3295,Taaach! Recordings,Taaach!\r\nEdition Chrome\r\nPostfach 10 50 54...,German Trance Label\r\nDistributed by InterGro...,None,28,2001,1998
4,3824,Fuse,Fuse\r\nbkb Dance Department gmbh\r\nAuhofstra...,German Trance label\r\nDistributed by Music Ma...,Massive Records,32,2001,1997
...,...,...,...,...,...,...,...,...
422,2731304,ELUSIVE INTELLIGENCE,childhoodintelligenceworldwide@gmail.com,None,Childhood Intelligence,0,0,0
423,2733764,Axe & Trap Studios,Underwood Business Park\r\nWookey Hole Road\r\...,"Recording studio located in the city of Wells,...",None,0,0,0
424,2786228,Εν Πλω,"Κολοκοτρώνη 49, 105 60 Αθήνα\r\n+302103226343 ...","Publishing company for books, music etc..",None,0,0,0
425,2875724,Doppelkirche Bonn-Schwarzrheindorf,Doppelkirche St. Maria und St. Clemens\r\nDixs...,Catholic parish church in Romanesque style fro...,None,0,0,0


In [ ]:
# Load Data
Load the new table into a postgres table named labels with the following columns:
id
name
contactinfo
profile
parentLabel
num_of_releases
min_release_year
max_release_year


In [10]:

# Function to load the transformed DataFrame into PostgreSQL
def load(df):
    print(f"{time.strftime('%H:%M:%S')}|Successful load of labels table into PostgreSQL")
    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(
        host='localhost',
        port='5432',
        dbname='postgres',
        user='postgres',
        password='welcome123'
    )

    # Create the labels table if it doesn't exist
    create_table_query = """
    CREATE TABLE IF NOT EXISTS labels (
        id SERIAL PRIMARY KEY,
        name VARCHAR(255),
        contactinfo TEXT,
        profile TEXT,
        parentLabel VARCHAR(255),
        num_of_releases INTEGER,
        min_release_year INTEGER,
        max_release_year INTEGER
    )
    """
    with conn.cursor() as cursor:
        cursor.execute(create_table_query)
        conn.commit()

    # Load the transformed DataFrame into the labels table
    with conn.cursor() as cursor:
        for _, row in df.iterrows():
            insert_query = """
            INSERT INTO labels (name, contactinfo, profile, parentLabel, num_of_releases, min_release_year, max_release_year)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            """
            values = (
                row['name'],
                row['contactinfo'],
                row['profile'],
                row['parentLabel'],
                row['num_of_releases'],
                row['min_release_year'],
                row['max_release_year']
            )
            cursor.execute(insert_query, values)
        conn.commit()
    print('Data loaded!')    
    conn.close()
obje = load(try2)


13:05:10|Successful load of labels table into PostgreSQL
Data loaded!


In [11]:
# Main function to orchestrate the ETL process
def main():
    df = extract()
    df = transform(df)
    load(df)

if __name__ == '__main__':
    main()

Begin reading new label releases from discogs.com
Successfully loaded 1946246 labels into DataFrame
Begin cleaning columns
13:06:53|Begin API calls for additional label info (1 labels)
13:06:55|Begin API calls for additional label info (1 labels)
13:06:58|Begin API calls for additional label info (1 labels)
13:07:00|Begin API calls for additional label info (1 labels)
13:07:02|Begin API calls for additional label info (1 labels)
13:07:04|Begin API calls for additional label info (1 labels)
13:07:06|Begin API calls for additional label info (1 labels)
13:07:08|Begin API calls for additional label info (1 labels)
13:07:11|Begin API calls for additional label info (1 labels)
13:07:13|Begin API calls for additional label info (1 labels)
13:07:15|Begin API calls for additional label info (1 labels)
13:07:18|Begin API calls for additional label info (1 labels)
13:07:20|Begin API calls for additional label info (1 labels)
13:07:22|Begin API calls for additional label info (1 labels)
13:07:24|

13:09:31|Failed API calls for labels: 30640
13:09:31|Begin API calls for additional label info (1 labels)
13:09:31 | Failed API call for label 31387
13:09:33|Failed API calls for labels: 31387
13:09:33|Begin API calls for additional label info (1 labels)
13:09:33 | Failed API call for label 31616
13:09:35|Failed API calls for labels: 31616
13:09:35|Begin API calls for additional label info (1 labels)
13:09:35 | Failed API call for label 31723
13:09:37|Failed API calls for labels: 31723
13:09:37|Begin API calls for additional label info (1 labels)
13:09:38 | Failed API call for label 33443
13:09:40|Failed API calls for labels: 33443
13:09:40|Begin API calls for additional label info (1 labels)
13:09:40 | Failed API call for label 33579
13:09:42|Failed API calls for labels: 33579
13:09:42|Begin API calls for additional label info (1 labels)
13:09:42 | Failed API call for label 34271
13:09:44|Failed API calls for labels: 34271
13:09:44|Begin API calls for additional label info (1 labels)


13:11:32|Failed API calls for labels: 82680
13:11:32|Begin API calls for additional label info (1 labels)
13:11:32 | Failed API call for label 84175
13:11:34|Failed API calls for labels: 84175
13:11:34|Begin API calls for additional label info (1 labels)
13:11:34 | Failed API call for label 84691
13:11:36|Failed API calls for labels: 84691
13:11:36|Begin API calls for additional label info (1 labels)
13:11:36 | Failed API call for label 84829
13:11:38|Failed API calls for labels: 84829
13:11:38|Begin API calls for additional label info (1 labels)
13:11:39 | Failed API call for label 86130
13:11:41|Failed API calls for labels: 86130
13:11:41|Begin API calls for additional label info (1 labels)
13:11:41 | Failed API call for label 86362
13:11:43|Failed API calls for labels: 86362
13:11:43|Begin API calls for additional label info (1 labels)
13:11:43 | Failed API call for label 87599
13:11:45|Failed API calls for labels: 87599
13:11:45|Begin API calls for additional label info (1 labels)


13:13:33|Failed API calls for labels: 187159
13:13:33|Begin API calls for additional label info (1 labels)
13:13:33 | Failed API call for label 188105
13:13:35|Failed API calls for labels: 188105
13:13:35|Begin API calls for additional label info (1 labels)
13:13:35 | Failed API call for label 192475
13:13:37|Failed API calls for labels: 192475
13:13:37|Begin API calls for additional label info (1 labels)
13:13:37 | Failed API call for label 193726
13:13:40|Failed API calls for labels: 193726
13:13:40|Begin API calls for additional label info (1 labels)
13:13:40 | Failed API call for label 195143
13:13:42|Failed API calls for labels: 195143
13:13:42|Begin API calls for additional label info (1 labels)
13:13:42 | Failed API call for label 196567
13:13:44|Failed API calls for labels: 196567
13:13:44|Begin API calls for additional label info (1 labels)
13:13:44 | Failed API call for label 198453
13:13:46|Failed API calls for labels: 198453
13:13:46|Begin API calls for additional label inf

13:15:34|Failed API calls for labels: 348638
13:15:34|Begin API calls for additional label info (1 labels)
13:15:34 | Failed API call for label 351479
13:15:36|Failed API calls for labels: 351479
13:15:36|Begin API calls for additional label info (1 labels)
13:15:36 | Failed API call for label 361078
13:15:38|Failed API calls for labels: 361078
13:15:39|Begin API calls for additional label info (1 labels)
13:15:39 | Failed API call for label 361619
13:15:41|Failed API calls for labels: 361619
13:15:41|Begin API calls for additional label info (1 labels)
13:15:41 | Failed API call for label 378105
13:15:43|Failed API calls for labels: 378105
13:15:43|Begin API calls for additional label info (1 labels)
13:15:43 | Failed API call for label 380955
13:15:45|Failed API calls for labels: 380955
13:15:45|Begin API calls for additional label info (1 labels)
13:15:45 | Failed API call for label 382365
13:15:47|Failed API calls for labels: 382365
13:15:47|Begin API calls for additional label inf

13:17:35|Failed API calls for labels: 630118
13:17:35|Begin API calls for additional label info (1 labels)
13:17:35 | Failed API call for label 644713
13:17:37|Failed API calls for labels: 644713
13:17:37|Begin API calls for additional label info (1 labels)
13:17:37 | Failed API call for label 649914
13:17:39|Failed API calls for labels: 649914
13:17:39|Begin API calls for additional label info (1 labels)
13:17:39 | Failed API call for label 652085
13:17:41|Failed API calls for labels: 652085
13:17:41|Begin API calls for additional label info (1 labels)
13:17:42 | Failed API call for label 660151
13:17:44|Failed API calls for labels: 660151
13:17:44|Begin API calls for additional label info (1 labels)
13:17:44 | Failed API call for label 662512
13:17:46|Failed API calls for labels: 662512
13:17:46|Begin API calls for additional label info (1 labels)
13:17:46 | Failed API call for label 662927
13:17:48|Failed API calls for labels: 662927
13:17:48|Begin API calls for additional label inf

13:19:38|Failed API calls for labels: 1010433
13:19:38|Begin API calls for additional label info (1 labels)
13:19:38 | Failed API call for label 1037282
13:19:40|Failed API calls for labels: 1037282
13:19:40|Begin API calls for additional label info (1 labels)
13:19:41 | Failed API call for label 1040376
13:19:43|Failed API calls for labels: 1040376
13:19:43|Begin API calls for additional label info (1 labels)
13:19:44 | Failed API call for label 1041572
13:19:46|Failed API calls for labels: 1041572
13:19:46|Begin API calls for additional label info (1 labels)
13:19:47 | Failed API call for label 1049898
13:19:49|Failed API calls for labels: 1049898
13:19:49|Begin API calls for additional label info (1 labels)
13:19:49 | Failed API call for label 1052400
13:19:51|Failed API calls for labels: 1052400
13:19:51|Begin API calls for additional label info (1 labels)
13:19:52 | Failed API call for label 1070050
13:19:54|Failed API calls for labels: 1070050
13:19:54|Begin API calls for additio

13:21:40|Failed API calls for labels: 1795801
13:21:40|Begin API calls for additional label info (1 labels)
13:21:41 | Failed API call for label 1798608
13:21:43|Failed API calls for labels: 1798608
13:21:43|Begin API calls for additional label info (1 labels)
13:21:43 | Failed API call for label 1800956
13:21:45|Failed API calls for labels: 1800956
13:21:45|Begin API calls for additional label info (1 labels)
13:21:45 | Failed API call for label 1806978
13:21:47|Failed API calls for labels: 1806978
13:21:47|Begin API calls for additional label info (1 labels)
13:21:48 | Failed API call for label 1844488
13:21:50|Failed API calls for labels: 1844488
13:21:50|Begin API calls for additional label info (1 labels)
13:21:50 | Failed API call for label 1858957
13:21:52|Failed API calls for labels: 1858957
13:21:52|Begin API calls for additional label info (1 labels)
13:21:52 | Failed API call for label 1867128
13:21:54|Failed API calls for labels: 1867128
13:21:54|Begin API calls for additio